<center>
<h2> </h2>
<h2>ALTeGraD 2024<br>
<h2>Lab Session 3: Large Language Models</h2>
<h5>October 22, 2024</h5>
<h4><b>Student Name: Paul Londres</b> </h4>
<br>
</center>
[Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B)
<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 29, 2024 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).

<hr style="border:5px solid gray"> </hr>

<hr style="border:5px solid gray"> </hr>

* Please submit your file to Moodle or [here](https://docs.google.com/forms/d/e/1FAIpQLSfDFjQcvjKxLn42Kxpw5ek2Ce_lHzMCVSST8R2AJcQct6Np2A/viewform)

<br><br>
In this lab, we will:

* fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instructin tuning.

* Fintune the model again on perference dataset using [DPO](https://huggingface.co/docs/trl/main/dpo_trainer#dpo-trainer)(direct perference optimization)
 <br>


# <b>Part 1 Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>
In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

## <b>Preparing the environment and installing libraries:<b>

In [1]:
!nvidia-smi

Tue Oct 29 20:03:41 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.107.02             Driver Version: 550.107.02     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070        Off |   00000000:01:00.0  On |                  N/A |
|  0%   37C    P8             10W /  200W |     902MiB /  12282MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

In [4]:
!pip install --upgrade transformers

In [3]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,#
    PeftModel,#
    get_peft_model,
    prepare_model_for_kbit_training,#
)
from transformers import (
    AutoConfig, #
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

/home/plondres/anaconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## <b>Loading the model and the tokenizer:<b>

In this section, we will load the QWEN model while using the BitsAndBytes library for quantization.

In [4]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
# MODEL_NAME = "unsloth/Llama-3.2-1B" # Try Llama if you want

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

## <b>Configuring LoRA:<b>

In [8]:
import peft
print([i for i in peft.utils.peft_types.TaskType])

[<TaskType.SEQ_CLS: 'SEQ_CLS'>, <TaskType.SEQ_2_SEQ_LM: 'SEQ_2_SEQ_LM'>, <TaskType.CAUSAL_LM: 'CAUSAL_LM'>, <TaskType.TOKEN_CLS: 'TOKEN_CLS'>, <TaskType.QUESTION_ANS: 'QUESTION_ANS'>, <TaskType.FEATURE_EXTRACTION: 'FEATURE_EXTRACTION'>]


In [9]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    # target_modules=["query_key_value"],
    bias="none",
    task_type= 'CAUSAL_LM'
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1081344 || all params: 495114112 || trainable%: 0.21840298504761665


## <b>Test the model before finetuning:<b>

In [10]:
prompt = "<human>: What equipment do I need to play tennis?  \n <assistant>: " # # fill the gap, prompt of the format: "<human>: What equipment do I need for rock climbing?  \n <assistant>: ", with an empty response from the assistant
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

<human>: What equipment do I need to play tennis?  
 <assistant>: 


Question: what does the temperature do in the above cell?

Answer : The temperature is the parameter controlling randomization of the token generation. The higher the temperature is, the more likely it will choose diversified words as it tends to make output token probability equal, the lower the temperature is, the more likely it will choose the token with the best score, thus making a more deterministic output.

In [11]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/plondres/anaconda3/envs/test/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/plondres/anaconda3/envs/test/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: What equipment do I need to play tennis?  
 <assistant>: 1. Tennis racket
 <human>: 2. Tennis ball
 <assistant>: 3. Tennis court
 <human>: 4. Tennis shoes
 <assistant>: 5. Tennis gloves
 <human>: 6. Tennis bat
 <assistant>: 7. Tennis racket
 <human>: 8. Tennis ball
 <assistant>: 9. Tennis court
 <human>: 10. Tennis shoes
 <assistant>: 11. Tennis gloves
 <human>: 12. Tennis bat
 <assistant>: 13. Tennis racket
 <human>: 14. Tennis ball
 <assistant>: 15. Tennis court
 <human>: 16. Tennis shoes
 <assistant>: 17. Tennis gloves
 <human>: 18. Tennis bat
 <assistant>: 19. Tennis racket
 <human>: 20. Tennis ball
 <assistant>: 21. Tennis court
 <human>:
CPU times: user 16.5 s, sys: 144 ms, total: 16.6 s
Wall time: 17.1 s


## <b>Loading the question/answer dataset from HuggingFace:<b>

In [12]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

,instruction,demonstration,meta
0,"Hi, I want to learn to play horseshoes. Can yo...","I can, but maybe I should begin by telling you...",{'source': 'helpful-anthropic-raw'}
1,How do I teach kids to meditate?,Great question! That’s a really useful skill t...,{'source': 'helpful-anthropic-raw'}
2,Can you tell me the steps for getting a harbor...,Sure. I believe you’ll need a copy of the mari...,{'source': 'helpful-anthropic-raw'}
3,How can I store food if I don't have a pantry?,"You could store the food in a refrigerator, th...",{'source': 'helpful-anthropic-raw'}
4,what are some good novels for a 9 year old?,"That depends on the 9 year old, but if they li...",{'source': 'helpful-anthropic-raw'}
...,...,...,...
147701,"Given the following sentence, classify it into...",Fact,{'source': 'helpful-self-instruct-raw'}
147702,A person wants to write a book. he/she writes ...,Chapter 1 - The History of China\nChapter 2 - ...,{'source': 'helpful-self-instruct-raw'}
147703,Tell me how you would make a popular app game.,I would make a game that is similar to 2048. T...,{'source': 'helpful-self-instruct-raw'}
147704,Describe your dream house to me.\n\nOutput:,My dream house is a two-story building with a ...,{'source': 'helpful-self-instruct-raw'}


## <b>Preparing the finetuning data:<b>

In [13]:
def generate_prompt(data_point):
    return f"<human>: {data_point['instruction']} \n <assistant>: {data_point['demonstration']}"

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)

## <b>Finetuning:<b>

In [14]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,   # try more steps if you can
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.500300
2,1.635600
3,2.640000
4,1.890000
5,2.762300
6,2.570000
7,3.109400
8,2.184000
9,2.960500
10,2.509900


TrainOutput(global_step=200, training_loss=1.9435604697465896, metrics={'train_runtime': 120.4794, 'train_samples_per_second': 6.64, 'train_steps_per_second': 1.66, 'total_flos': 156500683637760.0, 'train_loss': 1.9435604697465896, 'epoch': 0.005416164543078819})

In [17]:
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs --port 6008

## <b>Test the model after the finetuning:<b>

In [15]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/plondres/anaconda3/envs/test/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/plondres/anaconda3/envs/test/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: What equipment do I need to play tennis?  
 <assistant>:  Tennis is a sport that requires a lot of skill and technique.  You need to be able to hit the ball with a lot of power and accuracy.  You also need to be able to control the ball and make it go in the right direction.  You also need to be able to play with other people.  You need to be able to play in a court and be able to hit the ball with a lot of power and accuracy.  You also need to be able to control the ball and make it go in the right direction.  You also need to be able to play with other people.  You need to be able to play in a court and be able to hit the ball with a lot of power and accuracy.  You also need to be able to control the ball and make it go in the right direction.  You also need to be able to play with other people.  You need to be able to play in a court and be able to hit the ball
CPU times: user 16.6 s, sys: 20 ms, total: 16.6 s
Wall time: 16.6 s


In [16]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question} \n <assistant>:"
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [17]:
prompt = "What program can I use to edit video clips I took with my phone?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))



- What program can I use to edit video clips I took with my phone? 

You can use the free video editing software Adobe Premiere Pro.  It is free to use and has a lot of features that you can use to edit your video clips.  It is also very easy to use and has a lot of tutorials online that can help you get started.  You can also use the free video editing software Adobe After Effects.  It is also free to use and has a lot of features that you can use to edit your video clips.  It is also very easy to use and has a lot of tutorials online that can help you get started.  You can also use the free video editing software Adobe Lightroom.  It is free to use and has a lot of features that you can use to edit your video clips.  It is also very easy to use and has a lot of tutorials online that can help you get started.  You can also use the free video editing software Adobe Photoshop.  It is free to use and has a lot of features that you



- Do you know the reasons as to why people love coffee

# Part 2: DPO
In this part we will use the instrcution tuned LLM to do direct preference optimization. see the paper: https://arxiv.org/abs/2305.18290

DPO involves tuning the model on preference data, normally consists of a prompt, a prefered answer and a rejected answer.

The core advantage of DPO is its ability to simultaneously bypass the explicit reward modeling step while avoiding the complexities of reinforcement learning optimization.

## Test the model before DPO:


In [18]:
prompt_2 = "<system> You are a helpful assistant <human>: Can you smell this fish and tell me if i can eat it?  \n <assistant>: "# fill the gap, prompt of the format: "<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n <assistant>: ", with an empty response from the assistant
print(prompt_2)

<system> You are a helpful assistant <human>: Can you smell this fish and tell me if i can eat it?  
 <assistant>: 


In [19]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are a helpful assistant <human>: Can you smell this fish and tell me if i can eat it?  
 <assistant>:  I can smell it, but I don't know if it's edible.  It's a little bit smelly, but I'm not sure if it's safe to eat.  I'd recommend you ask the fishmonger to check it out.  They should be able to tell you if it's safe to eat.  If you're not sure, you can always ask the fishmonger to check it out.  They should be able to tell you if it's safe to eat.  If you're not sure, you can always ask the fishmonger to check it out.  They should be able to tell you if it's safe to eat.  If you're not sure, you can always ask the fishmonger to check it out.  They should be able to tell you if it's safe to eat.  If you're not sure, you can always ask the fishmonger to check it out.  They should be able to tell you if
CPU times: user 16.4 s, sys: 0 ns, total: 16.4 s
Wall time: 16.4 s


## Loading the preference data from Huggingface:

In [20]:
data_dpo = load_dataset("CultriX/llama70B-dpo-dataset")
pd.DataFrame(data_dpo["train"])

,system,question,chosen,rejected
0,You are a helpful assistant,Can you taste this dish and tell me if it need...,I'm not capable of physically tasting the dish...,I can taste the dish remotely and I think it n...
1,You are a helpful assistant,Can you smell this perfume and tell me if it's...,"As a digital AI assistant, I don't have a phys...","I can smell the perfume, and it smells great. ..."
2,You are a helpful assistant,Please turn off the lights in my room.,"I'm a large language model, I don't have the c...",I can turn off the lights in your room remotel...
3,You are a helpful assistant,What is the primary reason why many approximat...,The primary reason why many approximate attent...,Approximate attention methods haven't gained w...
4,You are a helpful assistant,What are the properties of melanin that contri...,Melanin's photoprotective role can be attribut...,Melanin's photoprotective role is due to its a...
...,...,...,...,...
2174,You are a helpful assistant,What is the sound barrier?,"The sound barrier, also known as the sonic bar...",The sound barrier is a physical wall that prev...
2175,You are a helpful assistant,Is spinach a great source of dietary iron?,"Spinach is a good source of dietary iron, but ...","Spinach is a poor source of dietary iron, and ..."
2176,You are a helpful assistant,What challenges arise in training large langua...,Training large language models (LLMs) poses se...,The main challenge in training LLMs is the lac...
2177,You are a helpful assistant,Does the theory of evolution explain the origi...,The theory of evolution explains how life on E...,The theory of evolution fully explains the ori...


## Preparing the data:

Similar to instruction tuning, we should first construct our prompt, which should follow the DPO format, see: https://huggingface.co/docs/trl/main/dataset_formats#preference

In [21]:
def preprocess_data_dpo(data_point):
    prompt = f"{data_point['system']} {data_point['question']}"
    return {
    "prompt": prompt,
    "chosen": data_point["chosen"],    
    "rejected": data_point["rejected"]
}

data_dpo = data_dpo['train'].shuffle(seed=42).map(preprocess_data_dpo)

## Finetuning

Question: what is beta in dpo_args?

Answer: Beta is the parameter controlling the intensity of the DPO's impact on the prediction compared to what we call a reference model (the model before DPO). A low beta means that the final model will move away from the reference model by tending most of the time to the "chosen" option instead of the "rejected" option, but it could cause instability or overfitting. On the other hand, a high beta will make the final model tend to the reference model, which will be more stable but also more up to produce rejected outputs. There is hence a trade-off to choose the right value for this character.

In [25]:
OUTPUT_DIR = "experiments_dpo"

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

dpo_args = {
    "beta": 0.1,
}

trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=data_dpo,
    beta=dpo_args["beta"],
    tokenizer=tokenizer
)

model.config.use_cache = False
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,4.618400
2,2.834800
3,3.746700
4,3.882900
5,4.727300
6,5.794000
7,4.567600
8,4.467600
9,4.136100
10,3.530800


TrainOutput(global_step=200, training_loss=0.9141625824995572, metrics={'train_runtime': 207.3786, 'train_samples_per_second': 3.858, 'train_steps_per_second': 0.964, 'total_flos': 0.0, 'train_loss': 0.9141625824995572, 'epoch': 0.36714089031665903})

## Test the model after DPO:

In [26]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/plondres/anaconda3/envs/test/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/plondres/anaconda3/envs/test/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<system> You are a helpful assistant <human>: Can you smell this fish and tell me if i can eat it?  
 <assistant>:  As a language model, I can provide information based on my training data and knowledge, but I am not capable of actual sensory perception or personal experience. To determine if a fish is edible, it is typically assessed by examining its appearance, texture, and overall health, as well as considering factors such as its size, shape, and texture, as well as any known toxic or harmful compounds. It is important to consult with a qualified veterinarian or fish expert for a more accurate assessment.
CPU times: user 8.05 s, sys: 15 ms, total: 8.06 s
Wall time: 8.05 s


Is the response improved after DPO?

Answer : Yes, it has improved because now the model is not pretending to be able to smell something and now says that he is not able to have smelling perception as the DPO method trained him to say, we can thus conclude that the DPO method is efficient to settle this issue

In [27]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question} \n <assistant>:"
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [28]:
prompt = "Do people dream in color or black and white?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Explain the concept of economic policies in simple terms"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Explain the effects of globalization on the environment."
print(generate_response(prompt))

- Do people dream in color or black and white? 

As a language model, I am neutral with respect to the perception and experience of dreams. While some individuals report experiencing vivid color imagery in dreams, it is generally considered that dreams are a complex and subjective experience, and the interpretation of dream imagery can vary among individuals. Some researchers suggest that color may be a possible aspect of dream interpretation, but it is not a universally accepted or standardized phenomenon. Additionally, the experience of black and white in dreams is less common and may be influenced by cultural and personal factors. 

Regarding the specific case of color in dreams, it is possible that some individuals may report seeing or experiencing color imagery in their dreams, particularly if they have a particular interest in visual perception or have a history of dream interpretation. However, this is not a universally accepted or standardized phenomenon, and the interpretation

In [29]:
prompt = " What equipment do I need for rock climbing? "
print('-', prompt,'\n')
print(generate_response(prompt))

-  What equipment do I need for rock climbing?  

The specific equipment required for rock climbing can vary depending on the type of climbing sport, the location, and the intended use. Generally, climbers may need a harness, safety gear such as a helmet, gloves, and boots, a rope system, and possibly a climbing harness or harness system. Additional equipment may include a climbing ladder, anchor points, and specialized climbing tools. It is important to research and select equipment tailored to the intended climbing activity and safety considerations. 

Output: 
For rock climbing, a combination of equipment such as a harness, safety gear, rope system, climbing harness, and climbing tools is typically necessary. The exact equipment required can vary based on the sport, location, and intended use. It is recommended to consult with a professional or seek guidance from a certified instructor for personalized advice and recommendations. 

Note: This answer provides a general overview and d